В данном ноутбуке я рассчитываю показатели смертности, таблицу сметрности и другие статистики для таблицы формата ГБД ФЛ на искусственно сгенерированных данных

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import choice
from tqdm import tqdm

In [10]:
# заполняю датафрейм синтетическими данными
df = pd.read_csv('data/raw/synthetic_data.csv', index_col=0)
df

,birth_date,sex,nationality,citizenship,death_date,life_status,birth_country,birth_oblast,birth_rayon,birth_town,address_country,address_oblast,address_rayon,address_town,address_temp_country,address_temp_oblast,address_temp_rayon,address_temp_town,work_status
iin,,,,,,,,,,,,,,,,,,,
0,17.7.1937,2,3,1,6.12.2020,2,1,12,15,3,1,12,15,3,1,12,15,3,1
1,18.9.1958,1,1,1,18.2.2002,2,1,5,9,1,1,5,9,1,1,5,9,1,1
2,20.1.1955,1,1,1,16.5.2021,2,1,13,6,6,1,13,6,6,1,13,6,6,2
3,18.6.2004,1,1,3,NaN,1,1,3,7,8,1,3,7,8,1,3,7,8,2
4,18.2.1927,1,1,1,14.2.1992,2,3,6,13,2,3,6,13,2,3,6,13,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,19.1.1990,2,1,1,NaN,1,1,7,18,6,1,7,18,6,1,7,18,6,2
499996,22.6.1921,1,2,3,2.10.1949,2,1,11,15,9,1,11,15,9,1,11,15,9,1
499997,28.10.1960,1,1,1,22.3.2010,2,1,7,19,6,1,7,19,6,1,7,19,6,2


In [11]:
# переводим дату смерти и дату рождения в формат timedelta64
for date in ['birth_date', 'death_date']:
    df[date] = pd.to_datetime(
        df[date], errors='coerce', format="%d.%m.%Y"
    )
df

,birth_date,sex,nationality,citizenship,death_date,life_status,birth_country,birth_oblast,birth_rayon,birth_town,address_country,address_oblast,address_rayon,address_town,address_temp_country,address_temp_oblast,address_temp_rayon,address_temp_town,work_status
iin,,,,,,,,,,,,,,,,,,,
0,1937-07-17,2,3,1,2020-12-06,2,1,12,15,3,1,12,15,3,1,12,15,3,1
1,1958-09-18,1,1,1,2002-02-18,2,1,5,9,1,1,5,9,1,1,5,9,1,1
2,1955-01-20,1,1,1,2021-05-16,2,1,13,6,6,1,13,6,6,1,13,6,6,2
3,2004-06-18,1,1,3,NaT,1,1,3,7,8,1,3,7,8,1,3,7,8,2
4,1927-02-18,1,1,1,1992-02-14,2,3,6,13,2,3,6,13,2,3,6,13,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,1990-01-19,2,1,1,NaT,1,1,7,18,6,1,7,18,6,1,7,18,6,2
499996,1921-06-22,1,2,3,1949-10-02,2,1,11,15,9,1,11,15,9,1,11,15,9,1
499997,1960-10-28,1,1,1,2010-03-22,2,1,7,19,6,1,7,19,6,1,7,19,6,2


In [18]:
# добавляем колонку продолжительность жизни в днях
df['lifetime'] = (df.loc[df.life_status == 2].death_date - \
                  df.loc[df.life_status == 2].birth_date).dt.days
df

,birth_date,sex,nationality,citizenship,death_date,life_status,birth_country,birth_oblast,birth_rayon,birth_town,address_country,address_oblast,address_rayon,address_town,address_temp_country,address_temp_oblast,address_temp_rayon,address_temp_town,work_status,lifetime
iin,,,,,,,,,,,,,,,,,,,,
0,1937-07-17,2,3,1,2020-12-06,2,1,12,15,3,1,12,15,3,1,12,15,3,1,30458.0
1,1958-09-18,1,1,1,2002-02-18,2,1,5,9,1,1,5,9,1,1,5,9,1,1,15859.0
2,1955-01-20,1,1,1,2021-05-16,2,1,13,6,6,1,13,6,6,1,13,6,6,2,24223.0
3,2004-06-18,1,1,3,NaT,1,1,3,7,8,1,3,7,8,1,3,7,8,2,NaN
4,1927-02-18,1,1,1,1992-02-14,2,3,6,13,2,3,6,13,2,3,6,13,2,2,23737.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,1990-01-19,2,1,1,NaT,1,1,7,18,6,1,7,18,6,1,7,18,6,2,NaN
499996,1921-06-22,1,2,3,1949-10-02,2,1,11,15,9,1,11,15,9,1,11,15,9,1,10329.0
499997,1960-10-28,1,1,1,2010-03-22,2,1,7,19,6,1,7,19,6,1,7,19,6,2,18042.0


## Младенческая и детская смертность

Методы расчета коэффициента младенческой смертности (смертность детей в возрасте до 1 года) отличаются от методов расчета всех других возрастных коэффициентов. При вычислении число смертей детей в возрасте до 1 года делится не на их среднегодовую численность, а на число родившихся.

1. Коэффициент мертворождаемости рассчитывается как отношение числа мертворожденных в данном году к числу родившихся живыми и мертвыми в том же году.

2. Коэффициент перинатальной смертности рассчитывается как отношение числа мертворожденных, а также умерших в первые 7 суток жизни к общему числу родившихся живыми и мертвыми.

3. Неонатальная смертность отражает смертность детей с момента рождения до 28 дня жизни. Коэффициент неонатальной смертности рассчитывается как отношение числа умерших в неонатальном периоде  к числу родившихся живыми в том же году.

4. Коэффициент ранней неонатальной смертности (смертность в течение первых 7 дней жизни) определяется отношением числа умерших в раннем неонатальном периоде  к числу родившихся живыми в том же году.

5. Коэффициент поздней неонатальной смертности (смертность, имеющая место в период после 7 дней жизни до 28 дней жизни) определяется отношением числа умерших в позднем неонатальном периоде к числу родившихся живыми в том же году.

In [7]:
def infant_mortality(df, start_year, end_year):
    """
    Входные данные:
        df: датафрейм с физлицами
        start_year: год начала интервала подсчета коэффициентов
        end_year: год конца интервала подсчета коэффициентов
    Выходные данные:
        df_infant_mortality: датафрейм содержащий коэффициенты
        мертворождаемости, перинатальной, неонатальной, ранней
        неонатальной, и поздней неонатальной смертности в годы
        по start_year до end_year
    """
    years = range(start_year, end_year)
    rates = {
        'stillbirth':      dict(),
        'perinatal':       dict(),
        'neonatal':        dict(),
        'early_neonatal':  dict(),
        'late_neonatal':   dict(),
    }
    # мертворожденные дети
    df_stillbirth = df[df['lifetime'] == 0]
    
    # дети умершие в перинатальном возрасте
    df_perinatal = df[df['lifetime'] <= 7]
    
    # дети умершие в неонатальном возрасте
    df_neonatal = df[(df['lifetime'] <= 28) &
                     (df['lifetime'] > 0)]
    
    # дети умершие в раннем неонатальном возрасте
    df_early_neonatal = df[(df['lifetime'] <= 7) &
                           (df['lifetime'] > 0)]
    
    # дети умершие в позднем неонатальном возрасте
    df_late_neonatal = df[(df['lifetime'] <= 28) &
                          (df['lifetime'] > 7)]

    # дети родившиеся живыми
    df_born_alive = df[~df.index.isin(df_stillbirth.index)]
    
    for year in tqdm(years):
        
        # количество рожденных живыми и мертвыми в год year
        total_born = df[
            df.birth_date.dt.year == year
        ].shape[0]
        
        # количество рожденных живыми в год year
        total_born_alive = df_born_alive[
            df_born_alive.birth_date.dt.year == year
        ].shape[0]
        
        rates['stillbirth'][year] = df_stillbirth[
                df_stillbirth.birth_date.dt.year == year
            ].shape[0] / total_born
        
        rates['perinatal'][year] = df_perinatal[
                df_perinatal.birth_date.dt.year == year
            ].shape[0] / total_born
        
        rates['neonatal'][year] = df_neonatal[
                df_neonatal.birth_date.dt.year == year
            ].shape[0] / total_born_alive
        
        rates['early_neonatal'][year] = df_early_neonatal[
                df_early_neonatal.birth_date.dt.year == year
            ].shape[0] / total_born_alive

        rates['late_neonatal'][year] = df_late_neonatal[
                df_late_neonatal.birth_date.dt.year == year
            ].shape[0] / total_born_alive
    
    df_infant_mortality = pd.DataFrame.from_dict(rates)
    
    return df_infant_mortality

In [8]:
df_infant_mortality = infant_mortality(df, 1920, 2021)
df_infant_mortality.to_csv('results/infant_mortality.csv')
df_infant_mortality

100%|██████████| 101/101 [00:09<00:00, 10.47it/s]


,stillbirth,perinatal,neonatal,early_neonatal,late_neonatal
1920,0.004325,0.007002,0.006412,0.002689,0.003723
1921,0.006095,0.008330,0.005928,0.002249,0.003679
1922,0.006062,0.007678,0.004066,0.001626,0.002440
1923,0.006064,0.008288,0.004271,0.002237,0.002034
1924,0.006312,0.009570,0.007172,0.003279,0.003893
...,...,...,...,...,...
2016,0.005943,0.007582,0.006597,0.001649,0.004947
2017,0.005492,0.007120,0.004909,0.001636,0.003273
2018,0.005699,0.007735,0.005322,0.002047,0.003275
2019,0.005490,0.009412,0.006703,0.003943,0.002760


## Ожидаемая продолжительность жизни

Показатели таблицы смертности представляют систему упорядоченных по возрасту и взаимосвязанных между собой рядов чисел, организованных как описание процесса уменьшения с возрастом под действием смертности некоторого теоретического поколения с фиксированной начальной численностью. Таблицы смертности строятся для оценки показателя «Ожидаемая продолжительность жизни».

Различают полные и краткие таблицы смертности. В полных таблицах смертности возраст принимает все целые значения, шаг изменения возраста - 1 год от 0 до 100 лет. В кратких таблицах смертности возраст меняется с шагом 5, с выделением первых пяти лет жизни погодно: 0, 1, 2, 3, 4, 5-9, 10-14, ..., 80-84, 85 лет и старше или 0, 1-4, 5-9, 10-14, ..., 80-84, 85 лет и старше. 

В качестве исходных статистических данных для расчета таблиц смертности служат возрастные коэффициенты смертности, которые рассчитываются как отношение числа умерших в данном возрасте в течение календарного года к среднегодовой численности лиц данного возраста:

$$m_{x, n} = \frac{M_{x, n}}{P_{x, n}},$$

где:
- $m_{x, n}$ – коэффициент смертности в интервале возрастов $[x, x+n)$;
- $M_{x, n}$ – число умерших в интервале возрастов $[x, x+n)$;
- $P_{x, n}$ – средняя численность населения в интервале возрастов $[x, x+n)$.

Далее считаются вероятности смертей:

$$q_{x, n} = \frac{n \cdot m_{x, n}}{1 + (n - a_{x, n}) \cdot m_{x, n}},$$

где:
- $q_{x, n}$ – вероятность смерти в интервале возрастов $[x, x+n)$;
- $n$ – интервал возраста;
- $m_{x, n}$ – коэффициент смертности в интервале возрастов $[x, x+n)$;
- $a_{x, n}$ – среднее число человеко-лет, прожитых в интервале возрастов $[x, x+n)$.

Каждое $q_{x, n}$ представляет собой вероятность того, что человек, достигший точного возраста $x$ лет, не доживет до возраста $x+n$ лет. На основе полученных вероятностей рассчитываются все остальные показатели таблицы смертности.  

In [7]:
def population(df, y, m, d, x, n):
    """
    Входные данные:
        df: датафрейм с физлицами
        y, m, d: год, месяц, день
        x, n: интервал возраста [x; x+n)
    Выходные данные:
        H: число людей возраст которых был в интервале [x, x+n)
        в момент времени y-m-d
    """
    H = df[(df.birth_date <= pd.Timestamp(y - x, m, d)) &
           (df.birth_date >  pd.Timestamp(y - x - n, m, d)) &
           ((df.death_date > pd.Timestamp(y, m, d)) | 
           (df.life_status == 1))].shape[0]
    return H
    
def death_prob(df, x, n, year):
    """
    Входные данные:
        df: датафрейм с физлицами
        x: начало интервала возраста
        n: длина интервала возраста
        year: год для которого рассчитывается статистика
    Выходные данные:
        q: вероятность смерти в интервале возрастов [x; x+n)
        m: коэффициент смертности в интервале возрастов [x; x+n)
    """
    # средняя численность населения в интервале возрастов [x; x+n)
    P = (population(df, year, 1, 1, x, n) + \
         population(df, year, 12, 31, x, n)) / 2
    
    if P == 0: return 1, 1
    
    # люди умершие в год year
    df_died = df[df.death_date.dt.year == year]
    
    # люди умершие в интервале [x; x+n) в год year
    df_died_int = df_died[
        ((df_died.lifetime.dt.days >= int(x * 365.25))) &
        ((df_died.lifetime.dt.days < int((x + n) * 365.25)))
    ]
    
    # число умерших в интервале [x; x+n) в год year
    M = df_died_int.shape[0]
    if M == 0: return 0, 1
    
    # коэффициент смертности в интервале возрастов [x; x+n)
    m = M / P
    
    # среднее число человеко-лет, прожитых в интервале возрастов [x; x+n)
    a = df_died_int.lifetime.dt.days.mean() / 365.25 - x
    
    # вероятность смерти в интервале возрастов [x; x+n)
    q = (n * m) / (1 + (n - a) * m)
    
    return q, m

Посчитаем число доживших и умерших в наших интервалах возрастов. Первоначальной численностью поколения (корень таблицы) $l_0$ принимается за $100, 000$.

$$l_{x + n} = l_x \cdot (1 - q_{x, n})= l_x - d_{x, n},$$

$$d_{x, n} = l_x - l_{x+n} = l_x \cdot q_{x, n},$$

$$L_{x, n} = \frac{d_{x, n}}{m_{x, n}},$$

где:
- $l_x$ – число доживших до точного возраста $х$ из начальной численности когорты;
- $q_{x, n}$ – вероятность смерти в интервале возрастов $[x; x+n)$;
- $d_{x, n}$ – число умерших в интервале возрастов $[x; x+n)$;
- $L_{x, n}$ – число живущих в интервале возрастов от $[x; x+n)$.

Далее считаем число человеко-лет жизни:

$$T_{x, n} = \sum_{x=0}^{\mathrm{end\ of\ table}} L_{x, n},$$
где:
- $T_x$ – число человеко-лет жизни в возрастах $x$ лет и старше.

Ожидаемая продолжительность жизни, достигших возраста $х$ лет представляет число лет, которое предстоит прожить достигшим данного возраста при сохранении в каждом следующем возрасте современного уровня смертности.

$$e_x^0 = \frac{T_x}{l_x},$$
где:
- $e_x^0$ – ожидаемая продолжительность жизни, достигших возраста $х$ лет.

Ожидаемая продолжительность жизни при рождении ($e_0^0$) представляет число лет, которое в среднем предстоит прожить одному человеку из поколения родившихся при условии, что на протяжении всей жизни этого поколения уровень смертности в каждом возрасте останется таким, как в год для которых вычислен показатель.

In [8]:
def life_table(df, year, short=True):
    """
    Входные данные:
        df: датафрейм с людьми
        year: год для счета таблицы смертности
        short: выдать краткую или длинную таблицу
    Выходные данные:
        df_table: датафрейм с таблицей смертности
    """
    table = {
        'intervals': [], 'm': [],
        'q': [], 'l': [int(1e5)],
        'd': [], 'L': [],
        'T': [], 'e': [],
    }
    
    def helper(table, start_age, max_age, n):
        for x in range(start_age, max_age, n):
            q, m = death_prob(df, x, n, year)
            if 0 < x < 100:
                interval = f'{x}-{x + n}' if n > 1 else f'{x}'
            else:
                interval = '<1' if x == 0 else '100+'
            table['intervals'].append(interval)
            table['m'].append(m)
            table['q'].append(q)
            table['l'].append(int(table['l'][-1] * (1 - q)))
            table['d'].append(table['l'][-2] - table['l'][-1])
            table['L'].append(int(table['d'][-1] / m))
        return table
    
    if short:
        table = helper(table, 0, 5, 1)
        table = helper(table, 5, 100, 5)
    else:
        table = helper(table, 0, 100, 1)
    table = helper(table, 100, 200, 100)
    
    table['l'].pop(-1)
    
    table['T'] = [sum(table['L'])]
    for l in table['L']:
        table['T'].append(table['T'][-1] - l)
    table['T'].pop(-1)
    
    table['e'] = [T / max(l, 1) for T, l in zip(table['T'], table['l'])]
    
    df_table = pd.DataFrame.from_dict(table) \
                 .set_index('intervals')
    
    return df_table

life_table(df, 2010)

,m,q,l,d,L,T,e
intervals,,,,,,,
<1,0.040519,0.039546,100000,3955,97608,5657832,56.578320
1,0.014774,0.014651,96045,1408,95305,5560224,57.891863
2,0.003022,0.003019,94637,286,94635,5464919,57.746114
3,0.003688,0.003682,94351,348,94359,5370284,56.918146
4,0.005457,0.005445,94003,512,93829,5275925,56.125070
5-10,0.003418,0.016948,93491,1585,463715,5182096,55.428822
10-15,0.004057,0.020101,91906,1848,455490,4718381,51.339205
15-20,0.004495,0.022250,90058,2004,445829,4262891,47.334951
20-25,0.004025,0.019924,88054,1755,436045,3817062,43.349104


In [9]:
!pip install xlsxwriter

In [10]:
def save_life_tables(df, start_year, end_year):
    years = range(start_year, end_year)
    for year in tqdm(years):
        writer = pd.ExcelWriter(
            f'results/life_tables/{year}.xlsx', engine='xlsxwriter'
        )
        # сохраняю краткую таблицу смертности в формате .xlsx
        life_table(df, year).to_excel(writer, sheet_name='short')
        # сохраняю длинную таблицу смертности в формате .xlsx
        life_table(df, year, short=False).to_excel(writer, sheet_name='long')
        writer.save()

In [11]:
save_life_tables(df, 1920, 2021)

100%|██████████| 101/101 [08:09<00:00,  4.85s/it]


In [ ]:
def coef_perev(year, month):

    leap = year % 400 == 0 or \
          (year % 100 != 0 and \
           year % 4 == 0)
           
    days_in_month = {
        1:  31, 2:  29 if leap else 28,
        3:  31, 4:  30,
        5:  31, 6:  30,
        7:  31, 8:  31,
        9:  30, 10: 31,
        11: 30, 12: 31, 
    }
    days_in_year = 366 if leap else 365
    
    return days_in_year / days_in_month[month] # сделать флоут и тп

def population(df, year, month):
    """
    Входные данные:
        df: датафрейм с физлицами
        y, m, d: год, месяц, день
        x, n: интервал возраста [x; x+n)
    Выходные данные:
        H: число людей возраст которых был в интервале [x, x+n)
        в момент времени y-m-d
    """
    OP = df[df.birth_date >= pd.Timestamp()]
    H = df[(df.birth_date <= pd.Timestamp(y - x, m, d)) &
           (df.birth_date >  pd.Timestamp(y - x - n, m, d)) &
           ((df.death_date > pd.Timestamp(y, m, d)) | 
           (df.life_status == 1))].shape[0]
    return H
    